<a href="https://colab.research.google.com/github/MWFK/TimeSeries_Sequence_with_TensorFlow/blob/main/8-%20MultiVariate_MultiOutput_MultiStep_TS_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definitions


**Multivariate time-series** 

has more than one time-dependent variable. Each variable depends not only on its past values but also has some dependency on other variables. This dependency is used for forecasting future values. In this case, there are multiple variables to be considered to optimally predict temperature.


**Multioutput regression**

involve predicting two or more numerical values given an input example. An example might be to predict a coordinate given an input, e.g. predicting x and y values.


**Multistep prediction**

is the task of predicting a sequence of values in a time series. A typical approach, known as multi-stage prediction, is to apply a predictive model step-by-step and use the predicted value of the current time step to determine its value in the next time step.

# Challenge

**TIME SERIES QUESTION**

Build and train a neural network to predict time indexed variables of
the multivariate house hold electric power consumption time series dataset.
Using a window of past 24 observations of the 7 variables, the model
should be trained to predict the next 24 observations of the 7 variables.

 ==============================================================================

**ABOUT THE DATASET**

Original Source:
https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption

The original 'Individual House Hold Electric Power Consumption Dataset'
has Measurements of electric power consumption in one household with
a one-minute sampling rate over a period of almost 4 years.

Different electrical quantities and some sub-metering values are available.

For the purpose of the examination we have provided a subset containing
the data for the first 60 days in the dataset. We have also cleaned the
dataset beforehand to remove missing values. The dataset is provided as a
csv file in the project.

The dataset has a total of 7 features ordered by time.

==============================================================================

**INSTRUCTIONS**

Complete the code in following functions:

1. windowed_dataset()
2. solution_model()

The model input and output shapes must match the following
specifications.

1. Model input_shape must be (BATCH_SIZE, N_PAST = 24, N_FEATURES = 7),
   since the testing infrastructure expects a window of past N_PAST = 24
   observations of the 7 features to predict the next 24 observations of
   the same features.

2. Model output_shape must be (BATCH_SIZE, N_FUTURE = 24, N_FEATURES = 7)

3. DON'T change the values of the following constants
   N_PAST, N_FUTURE, SHIFT in the windowed_dataset()
   BATCH_SIZE in solution_model() (See code for additional note on
   BATCH_SIZE).

4. Code for normalizing the data is provided - DON't change it.
   Changing the normalizing code will affect your score.

HINT: Your neural network must have a validation MAE of approximately 0.055 or less on the normalized validation dataset for top marks.

WARNING: Do not use lambda layers in your model, they are not supported
on the grading infrastructure.

WARNING: If you are using the GRU layer, it is advised not to use the
'recurrent_dropout' argument (you can alternatively set it to 0),
since it has not been implemented in the cuDNN kernel and may
result in much longer training times.


### Libs

In [1]:
import urllib
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import zipfile
import pandas as pd
import tensorflow as tf

### Data

In [2]:
# This function downloads and extracts the dataset to the directory that
# contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change https to http)
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('/content/household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()
        
download_and_extract_data()

In [3]:
df = pd.read_csv('/content/household_power_consumption.csv', sep=',',infer_datetime_format=True, index_col='datetime', header=0)
print(df.shape)
df.head()

(86400, 7)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


### Normalisation

In [4]:
# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

# Explanation of the Dataset Windowing

In [141]:
# ======================== expand_dims == Falqse ================================

# get the slices of an array in the form of objects 
# tf.data.Dataset.from_tensor_slices([[5, 10], [3, 6]]) will become == [5,10], [3,6]
ds = tf.data.Dataset.from_tensor_slices(df.head(12))
# tf.print(ds)
# print(list(ds.as_numpy_iterator()),'\n\n')


# ======================== expand_dims == True ================================

ds = tf.expand_dims(df.head(12), axis=-1)    
# tf.print(ds,'\n\n')

# get the slices of an array in the form of objects 
# tf.data.Dataset.from_tensor_slices([[5, 10], [3, 6]]) will become == [5,10], [3,6]
ds = tf.data.Dataset.from_tensor_slices(ds)
# tf.print(ds)
# list(ds.as_numpy_iterator())

In [142]:
ds = ds.window(size=6, shift=1, drop_remainder=True)
# tf.print(ds)

In [143]:
ds = ds.flat_map(lambda w: w.batch(6))
# tf.print(ds)
# tf.print(list(ds.as_numpy_iterator()))

In [144]:
ds = ds.map(lambda x : (x[:-7], x[-7:, :1])) 
tf.print(ds)
tf.print(list(ds.as_numpy_iterator()))

<MapDataset shapes: ((None, 7, 1), (None, 1, 1)), types: (tf.float64, tf.float64)>
[(array([], shape=(0, 7, 1), dtype=float64),
  array([[[4.216]],

       [[5.36 ]],

       [[5.374]],

       [[5.388]],

       [[3.666]],

       [[3.52 ]]])),
 (array([], shape=(0, 7, 1), dtype=float64),
  array([[[5.36 ]],

       [[5.374]],

       [[5.388]],

       [[3.666]],

       [[3.52 ]],

       [[3.702]]])),
 (array([], shape=(0, 7, 1), dtype=float64),
  array([[[5.374]],

       [[5.388]],

       [[3.666]],

       [[3.52 ]],

       [[3.702]],

       [[3.7  ]]])),
 (array([], shape=(0, 7, 1), dtype=float64),
  array([[[5.388]],

       [[3.666]],

       [[3.52 ]],

       [[3.702]],

       [[3.7  ]],

       [[3.668]]])),
 (array([], shape=(0, 7, 1), dtype=float64),
  array([[[3.666]],

       [[3.52 ]],

       [[3.702]],

       [[3.7  ]],

       [[3.668]],

       [[3.662]]])),
 (array([], shape=(0, 7, 1), dtype=float64),
  array([[[3.52 ]],

       [[3.702]],

       [[3.7  ]],

### Windowing Dataset

In [ ]:
# This function is used to map the unwindowed time series dataset to a
# windowed dataset so as to prepare it for training and validation.
# A window of features are constructed by shifting the window's starting
# position forward, one at a time (indicated by shift=1).
# For a window of 'n_past' number of observations of all time indexed variables in
# the dataset, the target for the window is the next 'n_future' number
# of observations of these variables, after the end of the window.

# COMPLETE THE CODE IN THE FOLLOWING FUNCTION.
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):

    ds = tf.expand_dims(series, axis=-1)         # Convert the 2D DataFrame to 3D Tensor => from (table, rows, columns) to (rows, columns, 1) ==> From (17097, 5) to (17097, 5, 1) 
    ds = tf.data.Dataset.from_tensor_slices(ds)  # Create a tensor => from (17097, 5, 1) to (5,1)

    # This line converts the dataset into a windowed dataset where a
    # window consists of both the observations to be included as features
    # and the targets.

    # Don't change the shift parameter. The test windows are
    # created with the specified shift and hence it might affect your
    # scores. Calculate the window size so that based on
    # the past 24 observations
    # (observations at time steps t=1,t=2,...t=24) of the 7 variables
    # in the dataset, you predict the next 24 observations
    # (observations at time steps t=25,t=26....t=48) of the 7 variables
    # of the dataset.

    # Hint: Each window should include both the past observations and
    # the future observations which are to be predicted. Calculate the
    # window size based on n_past and n_future.

    ds = ds.window(size=n_past + n_future,# YOUR CODE HERE,
                   shift=shift, drop_remainder=True)

    # flatten a dataset of batches into a dataset of their elements:
    # and convert it from tensors to a  a = {[1,2,3], [6,7,8], [10,11,36]} ==> a.flat_map(lambda x: Dataset.from_tensor_slices(x)) == {[1,2,3,6,7,8,10,11,36]}
    # https://www.tensorflow.org/api_docs/python/tf/data/Dataset?version=nightly#flat_map
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))

    # Now each window in the dataset has n_past and n_future observations.
    # This line maps each window to the form (n_past observations,
    # n_future observations) in the format needed for training the model.
    # Note: You can use a lambda function to map each window in the
    # dataset to it's respective (features, targets).

    # x[:-n_horizon]     == x[All batches except for the last one]  ==> X
    # x[-n_horizon:, :1] == x[The Last batch of features]           ==> y
    ds = ds.map(lambda x : (x[:-n_future], x[-n_future:, :1])) 
    # YOUR CODE HERE

    return ds.batch(batch_size).prefetch(1)

### Modeling

In [ ]:
# This function loads the data from csv file, normalizes the data and
# splits the dataset into train and validation data. It also uses the
# 'windowed_dataset' to split the data into windows of observations and
# targets. Finally it defines, compiles and trains a neural network.
# This function returns the trained model.

# COMPLETE THE CODE IN THE FOLLOWING FUNCTION.
def solution_model():
    # Downloads and extracts the dataset to the directory that contains this file.
    # download_and_extract_data()
    # Reads the dataset from the csv.
    df = pd.read_csv('/content/household_power_consumption.csv', sep=',',infer_datetime_format=True, index_col='datetime', header=0)

    # Number of features in the dataset. We use all features as predictors to
    # predict all features at future time steps.
    N_FEATURES = len(df.columns)

    # Normalizes the data
    data = df.values
    split_time = int(len(data) * 0.5)
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
    x_train = data[:split_time]
    x_valid = data[split_time:]

    # DO NOT CHANGE 'BATCH_SIZE' IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS
    # Explanation of the batch_size 
    # https://stackoverflow.com/questions/42998989/batch-size-in-tensorflow-understanding-the-concept

    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
    N_PAST = 24  # DO NOT CHANGE THIS

    # Number of future time steps which are to be predicted.
    N_FUTURE = 24  # DO NOT CHANGE THIS

    # By how many positions the window slides to create a new window
    # of observations.
    SHIFT = 1  # DO NOT CHANGE THIS

    # Code to create windowed train and validation datasets.
    # Complete the code in windowed_dataset.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)

    # Code to define your model.
    model = tf.keras.models.Sequential([

        # ADD YOUR LAYERS HERE.
        # Use this for random batch_size input_shape=(none, N_PAST, N_FEATURES) or 
        # input_shape=(BATCH_SIZE, N_PAST, N_FEATURES) for fixed batch size
        tf.keras.layers.Flatten(input_shape=(N_PAST, N_FEATURES)), 
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        #tf.keras.layers.Dense(n_horizon) https://www.kaggle.com/nicholasjhana/multi-variate-time-series-forecasting-tensorflow

        # If you don't follow the instructions in the following comments,
        # tests will fail to grade your code:
        # Whatever your first layer is, the input shape will be
        # (BATCH_SIZE, N_PAST = 24, N_FEATURES = 7)
        # The model must have an output shape of
        # (BATCH_SIZE, N_FUTURE = 24, N_FEATURES = 7).
        # Make sure that there are N_FEATURES = 7 neurons in the final dense
        # layer since the model predicts 7 features.

        # WARNING: If you are using the GRU layer, it is advised not to use the
        # 'recurrent_dropout' argument (you can alternatively set it to 0),
        # since it has not been implemented in the cuDNN kernel and may
        # result in much longer training times.
        tf.keras.layers.Dense(N_FEATURES)
    ])

    # Code to train and compile the model
    loss = tf.keras.losses.Huber()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)# YOUR CODE HERE
    model.compile(loss=loss, optimizer='adam', metrics=['mae'])
        # YOUR CODE HERE

    num_epochs = 10
    model.fit(x=train_set, validation_data=valid_set, epochs=num_epochs, verbose=2)
        # YOUR CODE HERE

    return model

### Execution

In [ ]:
# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.

if __name__ == '__main__':
    model = solution_model()
    model.save("model.h5")

Epoch 1/10
1349/1349 - 27s - loss: 0.0067 - mae: 0.0766 - val_loss: 0.0059 - val_mae: 0.0681 - 27s/epoch - 20ms/step
Epoch 2/10
1349/1349 - 26s - loss: 0.0047 - mae: 0.0646 - val_loss: 0.0048 - val_mae: 0.0604 - 26s/epoch - 19ms/step
Epoch 3/10
1349/1349 - 25s - loss: 0.0045 - mae: 0.0626 - val_loss: 0.0051 - val_mae: 0.0616 - 25s/epoch - 19ms/step
Epoch 4/10
1349/1349 - 25s - loss: 0.0044 - mae: 0.0616 - val_loss: 0.0048 - val_mae: 0.0596 - 25s/epoch - 18ms/step
Epoch 5/10
1349/1349 - 26s - loss: 0.0044 - mae: 0.0610 - val_loss: 0.0047 - val_mae: 0.0582 - 26s/epoch - 19ms/step
Epoch 6/10
1349/1349 - 25s - loss: 0.0043 - mae: 0.0606 - val_loss: 0.0046 - val_mae: 0.0583 - 25s/epoch - 18ms/step
Epoch 7/10
1349/1349 - 25s - loss: 0.0043 - mae: 0.0604 - val_loss: 0.0045 - val_mae: 0.0576 - 25s/epoch - 19ms/step
Epoch 8/10
1349/1349 - 25s - loss: 0.0043 - mae: 0.0601 - val_loss: 0.0045 - val_mae: 0.0582 - 25s/epoch - 18ms/step
Epoch 9/10
1349/1349 - 25s - loss: 0.0043 - mae: 0.0600 - val_lo